## Libraries

In [ ]:
import os
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
import rasterio as rio
from rioxarray.merge import merge_arrays

## The aim of this Notebook is to mosaic 4 tiles and clip them to AOI using long time series data

In [ ]:
# Set main directories and paths
main_dir = 'path_to_main_directory'
Clipped_path = os.path.join(main_dir, 'Clipped_FAPAR')
Boundary = os.path.join(main_dir, 'Boundary')
Tile_1 = os.path.join(main_dir, 'tile_1')
Tile_2 = os.path.join(main_dir, 'tile_2')
Tile_3 = os.path.join(main_dir, 'tile_3')
Tile_4 = os.path.join(main_dir, 'tile_4')
Images_in_tile1 = sorted(os.listdir(Tile_1))
Images_in_tile2 = sorted(os.listdir(Tile_2))
Images_in_tile3 = sorted(os.listdir(Tile_3))
Images_in_tile4 = sorted(os.listdir(Tile_4))
Copernicus_ex = rxr.open_rasterio(os.path.join(main_dir, 'example_copernicus_fapar.tif'), masked=True)

In [ ]:
# Load the shapefile
def load_shape_file(filepath):
    """Loads the shape file to create a mask for AOI clipping."""
    shpfile = gpd.read_file(filepath)
    print("Shapefile loaded. To prepare for masking, run the function `select_shape`.")
    return shpfile

# Create AOI mask from shapefile
def select_shape(shpfile):
    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]
    unioned_polygon = selected_rows.geometry.unary_union
    mask_polygon = gpd.GeoDataFrame(geometry=[unioned_polygon])
    print("Mask created.")
    return mask_polygon

shpfile = load_shape_file(os.path.join(Boundary, 'boundary_shapefile.shp'))
merged = select_shape(shpfile)

In [ ]:
# Set CRS of AOI mask based on Copernicus example
Copernicus_ex_crs = Copernicus_ex.rio.crs
merged.crs = Copernicus_ex_crs

In [ ]:
# Process each image in tiles and mosaic them
for i in range(len(Images_in_tile4)):
    tile1 = rxr.open_rasterio(os.path.join(Tile_1, Images_in_tile1[i]), masked=True)
    tile2 = rxr.open_rasterio(os.path.join(Tile_2, Images_in_tile2[i]), masked=True)
    tile3 = rxr.open_rasterio(os.path.join(Tile_3, Images_in_tile3[i]), masked=True)
    tile4 = rxr.open_rasterio(os.path.join(Tile_4, Images_in_tile4[i]), masked=True)
    
    # Mosaic tiles
    merged_raster = merge_arrays(dataarrays=[tile1, tile2, tile3, tile4])
    
    # Clip mosaic to AOI
    clipped = merged_raster.rio.clip(merged.geometry.apply(mapping), crs=merged.crs, all_touched=True, from_disk=True).squeeze()
    
    # Set directory for output clipped images by year
    year_dir = os.path.join(Clipped_path, Images_in_tile1[i][16:20])
    if not os.path.isdir(year_dir):
        os.makedirs(year_dir)
    
    # Save clipped image
    clipped.rio.to_raster(os.path.join(year_dir, Images_in_tile1[i] + '.tif'))